In [1]:
import jax
import jax.numpy as jnp

In [ ]:
train_features = jnp.load("train_features.npy")
train_lens = jnp.load("train_lens.npy")
train_min_len = 732


rng = jax.random.PRNGKey(6)
rng, val_rng = jax.random.split(rng, 2)

hold_out_idxs = jax.random.choice(val_rng, 1000, shape=(100,), replace=False)
mask = jnp.zeros(1000, dtype=bool).at[hold_out_idxs].set(True)

# Split the array into two arrays
val_features = train_features[mask]
train_features = train_features[~mask]

val_lens = train_lens[mask]
val_min_len = 73

In [ ]:
rng, new_rng, new_rng2, new_rng3 = jax.random.split(rng, num=4)

# print("Features shape: ", data.shape)
# print("Feature lengths shape: ", data_len.shape)

classes_idx = jax.random.choice(new_rng, data.shape[0], shape=[cats], replace=False)
classes_order = jax.random.permutation(new_rng2, cats)

query_idx = classes_order[-1]
query_class = classes_idx[query_idx]

features_mean = jnp.zeros((cats * examples_per_cat + 1, 512))
y = jnp.zeros((cats * examples_per_cat + 1))
idx = 0

data = data[:, :min_len, :]
# print(data.shape)
for i in range(cats - 1):
    rng, data_rng = jax.random.split(rng, num=2)
    label = classes_idx[classes_order[i]]
    cur_label = classes_order[i]

    idxs = jax.random.choice(data_rng, data.shape[1], shape=[examples_per_cat], replace=False).astype('int32')

    for j in range(examples_per_cat):
        features_mean = features_mean.at[idx, :].set(data[label, idxs[j], :])
        y = y.at[idx].set(cur_label)
        idx += 1

# query
rng, data_rng = jax.random.split(rng, num=2)
label = classes_idx[classes_order[-1]]
cur_label = classes_order[-1]
query_idxs = jax.random.choice(data_rng, data.shape[1], shape=[examples_per_cat + 1], replace=False).astype('int32')
for j in range(examples_per_cat):
    features_mean = features_mean.at[idx, :].set(data[label, query_idxs[j], :])
    y = y.at[idx].set(cur_label)
    idx += 1

# print(y.shape)
# print(y)

features_mean_y = jnp.concatenate([features_mean, jnp.expand_dims(y, axis=-1)], axis=-1)
features_mean_y_shuffled = jax.random.permutation(new_rng3, features_mean_y[:-1, :])
# print(features_mean_y_shuffled.shape)

features_mean = features_mean.at[:-1, :].set(features_mean_y_shuffled[:, :-1])

# print(y[:-1].shape)
# print(features_mean_y_shuffled[:, -1].shape)
y = y.at[:-1].set(features_mean_y_shuffled[:, -1])

features_mean = features_mean.at[-1, :].set(data[label, query_idxs[-1], :])
y = y.at[-1].set(cur_label)
y = y.squeeze()

# print(y)

x_query = features_mean[-1:, :]

v_data_full = jax.nn.one_hot(y, cats)
v_data = v_data_full[:, :-1] - 1 / cats * jnp.ones([1, cats - 1])

v_target_full = v_data_full[-1:, :]
v_target = v_data[-1:, :]

v_data_full = v_data_full[:-1, :]

query_init = jnp.zeros_like(v_target) - 1 / cats * jnp.ones([1, cats - 1])
v_data = v_data.at[-1, :].set(jnp.squeeze(query_init))

seq = jnp.concatenate([features_mean, x_query], axis=0)
target = x_query